## Lab 4: Pandas Continued

**This assignment should be completed by Sunday September 22, 2019 at 11:59 PM.**

We will continue discussion of [Pandas](https://pandas.pydata.org/). In this discussion, you will learn about:

* Grouping dataframes
* Merging dataframes

In [ ]:
# Setup from Lab 3

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import zipfile
from pathlib import Path
%matplotlib inline


def fetch_and_cache(data_url, file, data_dir="data", force=False):
    """
    Download and cache a url and return the file object.
    
    data_url: the web address to download
    file: the file in which to save the results.
    data_dir: (default="data") the location to save the data
    force: if true the file is always re-downloaded 
    
    return: The pathlib.Path to the file.
    """
    data_dir = Path(data_dir)
    data_dir.mkdir(exist_ok=True)
    file_path = data_dir/Path(file)
    if force and file_path.exists():
        file_path.unlink()
    if force or not file_path.exists():
        print('Downloading...', end=' ')
        resp = requests.get(data_url)
        with file_path.open('wb') as f:
            f.write(resp.content)
        print('Done!')
    else:
        import time 
        created = time.ctime(file_path.stat().st_ctime)
        print("Using cached version downloaded at", created)
    return file_path

data_url = 'https://www.ssa.gov/oact/babynames/state/namesbystate.zip'
namesbystate_path = fetch_and_cache(data_url, 'namesbystate.zip')
zf = zipfile.ZipFile(namesbystate_path, 'r')

column_labels = ['State', 'Sex', 'Year', 'Name', 'Count']

def load_dataframe_from_zip(zf, f):
    with zf.open(f) as fh: 
        return pd.read_csv(fh, header=None, names=column_labels)

states = [
    load_dataframe_from_zip(zf, f)
    for f in sorted(zf.filelist, key=lambda x:x.filename) 
    if f.filename.endswith('.TXT')
]

baby_names = states[0]
for state_df in states[1:]:
    baby_names = pd.concat([baby_names, state_df])
baby_names = baby_names.reset_index().iloc[:, 1:]
ca = baby_names[baby_names['State'] == 'CA']


## Data Aggregration (Grouping Data Frames)

### Question 1a
To count the number of instances of each unique value in a `Series`, we can use the `value_counts()` [method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html) as `df["col_name"].value_counts()`. Count the number of different names for each Year in `CA` (California).  (You may use the `ca` DataFrame created above.)

**Note:** *We are not computing the number of babies but instead the number of names (rows in the table) for each year.*

```
BEGIN QUESTION
name: q1a
```

In [ ]:
num_of_names_per_year = ... # TODO: solve me!
num_of_names_per_year[:5]

In [ ]:
# TEST
num_of_names_per_year[2007]
# EXPECTED: 7250

In [ ]:
# TEST
num_of_names_per_year[:5].sum()
# EXPECTED: 35612

In [ ]:
# TEST
num_of_names_per_year[1910]
# EXPECTED: 363

In [ ]:
# TEST
num_of_names_per_year[:15].sum()
# EXPECTED: 103752

### Question 1b
Count the number of different names for each gender in `CA`.


```
BEGIN QUESTION
name: q1b
```

In [ ]:
num_of_names_per_gender = # TODO: solve me!
num_of_names_per_gender

In [ ]:
# TEST
num_of_names_per_gender["F"]
# EXPECTED: 224810

In [ ]:
# TEST
num_of_names_per_gender["M"]
# EXPECTED: 156404

### Question 2: Groupby ###

Before we jump into using the [`groupby`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) function in Pandas, let's recap how grouping works in general for tabular data through a guided set of questions based on a small toy dataset of movies and genres. 

**Note:** If you want to see a visual of how grouping of data works, here is a link to an animation [Groupby Animation](http://www.ds100.org/sp18/assets/lectures/lec03/03-groupby_and_pivot.pdf)

**Problem Setting:** This summer 2018, there were a lot of good and bad movies that came out. Below is a dataframe with 5 columns: name of the movie as a `string`, the genre of the movie as a `string`, the first name of the director of the movie as a `string`, the average rating out of 10 on Rotten Tomatoes as an `integer`, and the total gross revenue made by the movie as an `integer`. The point of these guided questions (parts a and b) below is to understand how grouping of data works in general, **not** how grouping works in code. We will worry about how grouping works in Pandas in 7c, which will follow.

Below is the `movies` dataframe we are using, imported from the `movies.csv` file located in the `lab02` directory.

In [ ]:
movies = pd.read_csv("movies.csv")
movies

### Question 2a

If we grouped the `movies` dataframe above by `genre`, how many groups would be in the output and what would be the groups? Assign `num_groups` to the number of groups created (hard-code) and fill in `genre_list` as a list containing the names of genres as strings that represent the groups.

```
BEGIN QUESTION
name: q2a
```

In [ ]:
num_groups = ... # TODO: solve me!
genre_list = ... # TODO: solve me!

In [ ]:
# TEST
num_groups
# EXPECTED: 6

In [ ]:
# TEST
sorted(genre_list)
# EXPECTED: ['Action & Adventure', ..., 'Science Fiction & Fantasy']

### Question 2b

Whenever we group tabular data, it is usually the case that we need to aggregate values from the ungrouped column(s). If we were to group the `movies` dataframe above by `genre`, which column(s) in the `movies` dataframe would it make sense to aggregate if we were interested in finding how well each genre did in the eyes of people? Fill in `agg_cols` with the column name(s).

```
BEGIN QUESTION
name: q2b
```

In [ ]:
agg_cols = ... # TODO: solve me!

In [ ]:
# TEST
sorted(agg_cols)

Now, let's see `groupby` in action, instead of keeping everything abstract. To aggregate data in Pandas, we use the `.groupby()` [function](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html). The code below will group the `movies` dataframe by `genre` and find the average revenue and rating for each genre. You can verify you had the same number of groups as what you answered in 2a. 

In [ ]:
movies.loc[:, ['genre', 'rating', 'revenue']].groupby('genre').mean()

### Question 2c

Let's move back to baby names and specifically, the `ca` dataframe. Find the sum of `Count` for each `Name` in the `ca` table. You should use `df.groupby("col_name").sum()`. Your result should be a Pandas Series.

**Note:** *In this question we are now computing the number of registered babies with a given name.*

```
BEGIN QUESTION
name: q2c
```

In [ ]:
count_for_names = ... # TODO: solve me!
count_for_names.sort_values(ascending=False)[:5]

In [ ]:
# TEST
count_for_names["Michael"]
# EXPECTED: 431204

In [ ]:
# TEST
count_for_names[:100].sum()
# EXPECTED: 97322

In [ ]:
# TEST
count_for_names["David"]
# EXPECTED: 373116

In [ ]:
# TEST
count_for_names[:1000].sum()
# EXPECTED: 1344223

### Question 2d

Find the sum of `Count` for each female name after year 1999 (`>1999`) in California.


```
BEGIN QUESTION
name: q2d
```

In [ ]:
female_name_count = ... # TODO: solve me!
female_name_count.sort_values(ascending=False)[:5]

In [ ]:
# TEST
female_name_count["Emily"]
# EXPECTED: 49605

In [ ]:
# TEST
female_name_count[:100].sum()
# EXPECTED: 51322

In [ ]:
# TEST
female_name_count["Isabella"]
# EXPECTED: 47447

In [ ]:
# TEST
female_name_count[:10000].sum()
# EXPECTED: 4104589

### Question 3: Grouping Multiple Columns

Let's move back to the `movies` dataframe. Which of the following lines of code will output the following dataframe? Write your answer (hard-coded) as either 1, 2, 3, or 4. Recall that the arguments to `pd.pivot_table` are as follows: `data` is the input dataframe, `index` includes the values we use as rows, `columns` are the columns of the pivot table, `values` are the values in the pivot table, and `aggfunc` is the aggregation function that we use to aggregate `values`.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>rating</th>
      <th>5</th>
      <th>6</th>
      <th>7</th>
      <th>8</th>
    </tr>
    <tr>
      <th>genre</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Action &amp; Adventure</th>
      <td>208681866.0</td>
      <td>129228350.0</td>
      <td>318344544.0</td>
      <td>6708147.0</td>
    </tr>
    <tr>
      <th>Animation</th>
      <td>374408165.0</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Comedy</th>
      <td>55383976.0</td>
      <td>30561590.0</td>
      <td>NaN</td>
      <td>111705055.0</td>
    </tr>
    <tr>
      <th>Drama</th>
      <td>NaN</td>
      <td>17146165.5</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Horror</th>
      <td>NaN</td>
      <td>NaN</td>
      <td>68765655.0</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Science Fiction &amp; Fantasy</th>
      <td>NaN</td>
      <td>312674899.0</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
  </tbody>
</table>

1) `pd.pivot_table(data=movies, index='genre', columns='rating', values='revenue', aggfunc=np.mean)`

2) `movies.groupby(['genre', 'rating'])['revenue'].mean()`

3) `pd.pivot_table(data=movies, index='rating', columns='genre', values='revenue', aggfunc=np.mean)`

4) `movies.groupby('revenue')[['genre', 'rating']].mean()`


```
BEGIN QUESTION
name: q3
```

In [ ]:
q3_answer = ... # TODO: solve me!

In [ ]:
# TEST
q3_answer

### Question 4: Merging


#### Question 4a

Time to put everything together! Merge `movies` and `count_for_names` to find the number of registered baby names for each director using [`pd.merge`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html?highlight=merge#pandas.merge). Only include names that appear in both `movies` and `count_for_names`.

**Hint:** You might need to convert the `count_for_names` series to a dataframe. Take a look at the ``to_frame`` method of a series to do this. 

Your first row should look something like this:

**Note**: It is ok if you have 2 separate columns with names instead of just one column.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>director</th>
      <th>genre</th>
      <th>movie</th>
      <th>rating</th>
      <th>revenue</th>
      <th>Count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>David</td>
      <td>Action &amp; Adventure</td>
      <td>Deadpool 2</td>
      <td>7</td>
      <td>318344544</td>
      <td>371646</td>
    </tr>
  </tbody>
</table>
</table>


```
BEGIN QUESTION
name: q4a
```

In [ ]:
merged_df = ... # TODO: solve me!
merged_df.head()

In [ ]:
# TEST
merged_df.loc[0, 'Count']
# EXPECTED: 373116

In [ ]:
# TEST
merged_df.loc[3, 'Count']
# EXPECTED: 5


In [ ]:
# TEST
merged_df.loc[7, 'Count']
# EXPECTED: 7237

In [ ]:
# TEST
merged_df['Count'].sum()
# EXPECTED: 865131

In [ ]:
# TEST
len(merged_df)
# EXPECTED: 14

#### Question 4b

How many directors in the original `movies` table did not get included in the `merged_df` dataframe? Please hard-code your answer as a number in `q4b`, then explain your answer in 1-2 sentences as a comment below.


```
BEGIN QUESTION
name: q4b
```

In [ ]:
q_4b = ... # TODO: solve me!

# Explain your solution: ...

In [ ]:
# TEST
q_4b